    Iteractive widgets to accompany MRS360 Lectures
    Copyright (C) 2016  Simon Biggs

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU Affero General Public License as
    published by the Free Software Foundation, either version 3 of the
    License, or (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU Affero General Public License for more details.

    You should have received a copy of the GNU Affero General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

# Jupyter Notebook to accompany MRS360 Topic 1 Foundations
To use fiddle with the examples here first you need to click `Cell > Run All`. That will then allow you to use the sliders available in various examples. For further help on what to do here see http://jupyter.org/.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sympy
from ipywidgets import interact, fixed
from IPython.display import display, clear_output

sympy.init_printing()

## Demonstration of exponential approximation

In [ ]:
def approximate_exponential(x, number_of_terms):
    e = np.zeros_like(x)
    for i in range(number_of_terms):
        e += x ** i / np.math.factorial(i)
    
    return e

def aproximate_and_plot(number_of_terms=2):
    x_1 = np.linspace(-5, 5)
    y_actual = np.exp(x_1)
    
    y_approx = approximate_exponential(x_1, number_of_terms)
    
    plt.figure(figsize=(6,5.5))
    plt.plot(x_1, y_actual, 'r')
    plt.plot(x_1, y_approx, 'b--')
    

    plt.xlim([-5, 5])
    plt.ylim([-10, 15])

    plt.title('Approximation of an exponential')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid(True)
    
interact(aproximate_and_plot, number_of_terms = [1,14]);

## Example plotting the binomial approximation

In [ ]:
def plot_binomial_series_expansion(sympy_function_of_x, number_of_terms=2):
    clear_output(wait=True)
    x = sympy.symbols('x')
    series_expansion = sympy_function_of_x.series(x, 0, number_of_terms).removeO()
    
    numpy_function = sympy.lambdify(x, series_expansion)
    
    x_2 = np.linspace(-2, 2)
    x_left = np.linspace(-2, -1.00001)
    x_right = np.linspace(-0.99999, 2)
    y_actual_left = 1 / (1 + x_left)
    y_actual_right = 1 / (1 + x_right)
    
    y_approx = numpy_function(x_2)
    
    fig = plt.figure(figsize=(6,5.5))
    ax = fig.add_subplot(111)
    ax.plot(x_left, y_actual_left, 'r')
    ax.plot(x_right, y_actual_right, 'r')
    ax.plot(x_2, y_approx, 'b--')
    

    ax.set_xlim([-2, 2])
    ax.set_ylim([-10, 10])

    ax.set_title(r'Approximation of $\frac{1}{1+x}$')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.grid(True)
    
    plt.show(fig)
    display(series_expansion)


x = sympy.symbols('x') 
sympy_function_of_x = 1 / (1 + x)

interact(
    plot_binomial_series_expansion, 
    sympy_function_of_x=fixed(sympy_function_of_x), 
    number_of_terms = [2,20]);

## Approximating exponential decay

In [ ]:
e = sympy.functions.elementary.exponential.exp(1)
x = sympy.symbols('x')
mu = sympy.symbols('mu')

def plot_intensity_approximation(sympy_function, mu_value=0.2, number_of_terms=2):
    clear_output(wait=True)
    series_approximation = sympy_function.series(x, 0, number_of_terms).removeO()
    
    series_approximation_subs = series_approximation.subs([(mu, mu_value), (e, np.exp(1))])
    sympy_function_subs = sympy_function.subs([(mu, mu_value), (e, np.exp(1))])
    
    numpy_series_approximation = sympy.lambdify(x, series_approximation_subs)
    numpy_actual = sympy.lambdify(x, sympy_function_subs)
    
    next_term = sympy_function.series(x, 0, number_of_terms + 1).removeO() - series_approximation
    last_term = series_approximation - sympy_function.series(x, 0, number_of_terms - 1).removeO()
    
    point_of_reasonable_accuracy = last_term/10 + next_term
    point_of_reasonable_accuracy_subs = point_of_reasonable_accuracy.subs([(mu, mu_value), (e, np.exp(1))])
    
    x_value_of_reasonable_accuracy = sympy.solve(point_of_reasonable_accuracy_subs, x)[1]
    x_values = np.linspace(0.001, 10)
    
    y_actual = numpy_actual(x_values)
    y_approx = numpy_series_approximation(x_values)
    
    fig = plt.figure(figsize=(6,5.5))
    ax = fig.add_subplot(111)
    
    ax.plot(x_values, y_actual, 'r')
    ax.plot(x_values, y_approx, 'b--')
    ax.plot(
        [x_value_of_reasonable_accuracy, x_value_of_reasonable_accuracy], [0, 1], 'g--', 
        label='point at which the next term would be 10x smaller than the current last term')

    ax.set_xlim([0, 10])
    ax.set_ylim([0, 1])

    ax.set_title(r'Approximation of {}'.format(str(sympy_function)))
    ax.set_xlabel('x')
    ax.set_ylabel('y')  
    plt.grid(True)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                     box.width, box.height * 0.9])

    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
              fancybox=True, shadow=True, ncol=5)
    plt.show(fig)
    display(series_approximation)

sympy_function = e ** (-mu * x)

interact(
    plot_intensity_approximation, 
    sympy_function=fixed(sympy_function),
    mu_value = [0.1, 2.0],
    number_of_terms = [2,10]);